In [ ]:
# Somnolencia al Manejar

# [Víndulo del Kaggle](https://www.kaggle.com/datasets/ismailnasri20/driver-drowsiness-dataset-ddd)

import os
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
"""Dado el peso de las imágenes, debemos primero subir las imágenes a Google Drive y desde ahí modificarlas."""

drive.mount('/content/drive')

"""## Preparación

El set de imágenes se obtiene por medio de un `.zip`. `!unzip` permite descomprimir su contenido en el proyecto.
"""

!unzip /content/drive/MyDrive/dataset.zip #&> /dev/null

In [ ]:
"""Para poder simplificar su manejo, se modificarán las imágenes para tener solo las escala de grises. Los colores en las imágenes de este proyecto no son de mayor importancia, logrando descartarlas y así simplificar el proceso."""

non_drowsy = {
    'path': './non_drowsy/',
    'end_path' : 'grayscale_non_drowsy/',
    'files': os.listdir(path='./non_drowsy')[0:150],
    'target' : 0
}
drowsy =  {
    'path': './drowsy/',
    'end_path' : 'grayscale_drowsy/',
    'files': os.listdir(path='./drowsy')[0:150],
    'target' : 1
}

dataset = {
    'imagenes' : [],
    'data' : [],
    'target' : []
}

In [ ]:
def mostrar(n1, n2):
  img1 = cv2.imread(drowsy['path'] + drowsy['files'][n1])
  cv2_imshow(img1)
  print(img1.shape)

  img2 = cv2.imread(non_drowsy['path'] + non_drowsy['files'][n2])
  cv2_imshow(img2)
  print(img2.shape)

mostrar(6,10)



In [ ]:
def grayscale(set, esta_zzz):
    for file in set['files']:
        imagen = cv2.imread(set['path'] + file)
        gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

        # Este código es extremadamente temporal
        dataset['imagenes'].append(gray) # Imagenes
        dataset['data'].append(gray.ravel())
        dataset['target'].append(set['target']) #

        if not cv2.imwrite(set['end_path'] + file, gray):
            print(f'ERROR guardando {file} a {set["end_path"]}')
        else:
            print(f'Guardado {file} en {set["end_path"]}')

try:
  os.mkdir(non_drowsy['end_path'])
  os.mkdir(drowsy['end_path'])
except:
  print("Carpetas ya existen")

grayscale(drowsy, 1)
grayscale(non_drowsy, 0)



In [ ]:
def mostrar_gris(n1, n2):
  img1 = cv2.imread(drowsy['end_path'] + drowsy['files'][n1])
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
  cv2_imshow(img1)
  print(img1.shape)

  img2 = cv2.imread(non_drowsy['end_path'] + non_drowsy['files'][n2])
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
  cv2_imshow(img2)
  print(img2.shape)

mostrar_gris(6,10)

print(dataset['imagenes'][0])
print(dataset['data'][0])

print(dataset['imagenes'][len(dataset['data']) - 1])
print(dataset['data'][len(dataset['data']) - 1])

dataframe = pd.DataFrame.from_dict(dataset)
print(dataframe.head())

print(dataframe[['data']].describe())

In [ ]:
"""## Desarrollo"""

len_drowsy = len(os.listdir('./drowsy/'))
size_drowsy = os.path.getsize('./drowsy/')

len_non_drowsy = len(os.listdir('./non_drowsy/'))
size_non_drowsy = os.path.getsize('./non_drowsy/')

print('Cantidad de imágenes de somnolencia:', len_drowsy, f'({size_drowsy})')
print('Cantidad de imágenes de sin somnolencia:', len_non_drowsy, f'({size_non_drowsy})')
print('Total:', len_drowsy + len_non_drowsy, f'({size_drowsy + size_non_drowsy})')

len_gray_drowsy = len(os.listdir('./grayscale_drowsy/'))
size_gray_drowsy = os.path.getsize('./grayscale_drowsy/')

len_gray_non_drowsy = len(os.listdir('./grayscale_non_drowsy/'))
size_gray_non_drowsy = os.path.getsize('./grayscale_non_drowsy/')

print('Cantidad de imágenes de somnolencia:', len_gray_drowsy, f'({size_gray_drowsy })')
print('Cantidad de imágenes de sin somnolencia:', len_gray_non_drowsy, f'({size_gray_non_drowsy})')
print('Total:', len_gray_drowsy + len_gray_non_drowsy, f'({size_gray_drowsy + size_gray_non_drowsy})')

In [ ]:
figura = plt.figure()
for i in range(1,26):
  grid = figura.add_subplot(5,5,i)

  grid.imshow(dataset['imagenes'][i])

plt.show()

imagenes = dataset['imagenes']
features = dataset['data']
targets = dataset['target']

train_features, test_features, train_targets, test_targets = train_test_split(
        features, targets,
        train_size=0.8,
        test_size=0.2,
        random_state=43,
        stratify=targets
    )

classifier = SVC(kernel = 'linear')
classifier.fit(train_features, train_targets)

prediction_training_targets = classifier.predict(train_features)
self_accuracy = accuracy_score(train_targets, prediction_training_targets)
print("Precisión para información entrenada:", self_accuracy)


prediction_test_targets = classifier.predict(test_features)
test_accuracy = accuracy_score(test_targets, prediction_test_targets)
print("Precisión por información de testeo:", test_accuracy)

In [ ]:
"""# Test"""

test_target = []
path = './non_drowsy/'
imgs_drowsy = os.listdir(path)[800:1000]
for i in range(0,127):
  test_imagen = cv2.imread(path + imgs_drowsy[i])
  test_imagen = cv2.cvtColor(test_imagen, cv2.COLOR_BGR2GRAY)
  test_target.append(test_imagen.ravel())

resultado =classifier.predict(test_target)
print(resultado)

test_target = []
path = './drowsy/'
imgs_drowsy = os.listdir(path)[2000:2200]
for i in range(0,127):
  test_imagen = cv2.imread(path + imgs_drowsy[i])
  test_imagen = cv2.cvtColor(test_imagen, cv2.COLOR_BGR2GRAY)
  test_target.append(test_imagen.ravel())

resultado =classifier.predict(test_target)
print(resultado)